## Prediction de salaire

### Bibliothèques utiles

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
sns.set_context("talk")
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


### I-  Description de la base de donnée

#### Salaires dans le secteur privé selon le sexe et la catégorie socioprofessionnelle (base communale)

* Le champ correspond aux salariés du privé, y compris bénéficiaires de contrats aidés et de contrats de professionnalisation ; hors apprentis, stagiaires, salariés agricoles et salariés des particuliers employeurs.

* Les données sur les salaires au lieu de travail sont ventilées selon le sexe et la catégorie socioprofessionnelle (hors agriculture), et détaillées par territoire : commune, arrondissement municipal, arrondissement, aire d'attraction des villes 2020, bassin de vie 2022, établissement public de coopération intercommunal, unité urbaine 2020, zone d'emploi 2020, département, région, France hors Mayotte.

Variables explicatives retenues :

SEX : Sexe (Homme/Femme) - impact attendu sur l'écart salarial

PCS_ESE : Profession et Catégorie Socioprofessionnelle - déterminant principal

TIME_PERIOD : Année (2022-2023) - évolution temporelle

GEO : Code géographique - variations territoriales

In [18]:
## lien vers le dataset
data_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_data.csv"
metadata_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_metadata.csv"


In [19]:
data = pd.read_csv(data_path, sep=';')
metadata = pd.read_csv(metadata_path, sep=';')

In [25]:
# Affichage des informations de base
print(f" Dimensions du dataset: {data.shape}")
print(f" Colonnes disponibles: {list(data.columns)}")

 Dimensions du dataset: (370710, 9)
 Colonnes disponibles: ['GEO', 'GEO_OBJECT', 'FREQ', 'SEX', 'PCS_ESE', 'DERA_MEASURE', 'CONF_STATUS', 'TIME_PERIOD', 'OBS_VALUE']


In [26]:
print("\n Aperçu des premières lignes:")
print(data.head())


 Aperçu des premières lignes:
     GEO GEO_OBJECT FREQ SEX PCS_ESE                      DERA_MEASURE CONF_STATUS  TIME_PERIOD  OBS_VALUE
0  26362     BV2022    A   F      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,157.285
1  26324     BV2022    A  _T       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  3,112.938
2  26307     BV2022    A  _T       6  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,013.097
3  26362     BV2022    A  _T      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,483.037
4  27170     BV2022    A   F       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,107.221


In [23]:
print(" Types de données:")
print(data.dtypes)

 Types de données:
GEO              object
GEO_OBJECT       object
FREQ             object
SEX              object
PCS_ESE          object
DERA_MEASURE     object
CONF_STATUS      object
TIME_PERIOD       int64
OBS_VALUE       float64
dtype: object


In [24]:

print("\n📋 Informations détaillées:")
print(data.info())


📋 Informations détaillées:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370710 entries, 0 to 370709
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   GEO           370710 non-null  object 
 1   GEO_OBJECT    370710 non-null  object 
 2   FREQ          370710 non-null  object 
 3   SEX           370710 non-null  object 
 4   PCS_ESE       370710 non-null  object 
 5   DERA_MEASURE  370710 non-null  object 
 6   CONF_STATUS   370710 non-null  object 
 7   TIME_PERIOD   370710 non-null  int64  
 8   OBS_VALUE     350250 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 25.5+ MB
None


### II-  Analyse exploratoire de donnée

### III- Modèle de prédiction 